In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression as lp
from scipy import stats
import statsmodels.api as sm

In [2]:
dt = pd.read_csv("TP1H2021.csv", delimiter=None)
dt_2 = pd.read_csv("3tp.csv", delimiter=None)

In [3]:
dt.head(5)

,Rm_Rf,rSMB,rHML,rRMW,rCMA,Rs_Rf
0,3.65,0.16,-0.57,2.10,-2.45,5.81
1,-4.94,0.18,1.23,1.75,0.63,-5.01
2,4.24,-0.96,-1.25,1.60,-1.41,4.40
3,2.32,1.44,-0.83,0.87,-0.55,3.46
4,-1.59,1.56,-0.98,-1.81,0.16,-3.65


In [4]:
df = dt.to_numpy()
df_2 = dt_2.to_numpy()

In [5]:
#Création des vecteurs


y = df[0:200,0]
x = df[0:200, df.shape[1] -1]
x = x.reshape(-1, 1)
x2 = sm.add_constant(x)

In [6]:
#Question 1.1


#Avec intercept
model = sm.OLS(y, x2).fit()
print(f"Erreurs au carré: {round(model.mse_total, 4)}, R2: {round(model.rsquared_adj, 4)}, Intercept: {round(model.params[0], 4)}, Coeff: {round(model.params[1], 4)}")
#Sans intercept
m_wo_int = sm.OLS(y, x).fit()
print(f"Erreurs au carré: {round(m_wo_int.mse_total, 4)}, R2: {round(m_wo_int.rsquared_adj, 4)}, Intercept: 0, Coeff: {round(m_wo_int.params[0], 4)}")

Erreurs au carré: 15.9544, R2: 0.664, Intercept: 0.1469, Coeff: 0.882
Erreurs au carré: 16.2635, R2: 0.6708, Intercept: 0, Coeff: 0.8877


In [7]:
#Question 1.3


print(f"W/ int: {round(model.pvalues[0], 4)} > 0.05, on ne peut pas rejeter H0")

W/ int: 0.3756 > 0.05, on ne peut pas rejeter H0


In [8]:
#Question 1.4
print(f"La forme de la régression sera : u ** 2 = \u03b1 + (\u03b1Y ** 2)")

white = sm.stats.diagnostic.het_white(model.resid, x2)
lm = white[0]
print(f"{lm} < {stats.chi2.isf(0.05, 1)}, on ne peut pas rejeter H0, nous ne sommes pas en présence d'hétéroscédasticité")

La forme de la régression sera : u ** 2 = α + (αY ** 2)
2.1969805258782893 < 3.8414588206941285, on ne peut pas rejeter H0, nous ne sommes pas en présence d'hétéroscédasticité


In [9]:
#Question 1.5 régression linéaire

res_t_1 = sm.add_constant(model.resid[1:]) ** 2
breusch = sm.stats.diagnostic.het_breuschpagan(model.resid[0:model.resid.shape[0]-1], res_t_1)
p_value_lm = breusch[1]
print(f"{p_value_lm} < 0.05, on peut rejeter H0")

1.4996099404113279e-05 < 0.05, on peut rejeter H0


In [10]:
smb = np.atleast_2d(df[0:200, 1]).T
hml = np.atleast_2d(df[0:200, 2]).T
x_mat = sm.add_constant(np.hstack((x, smb, hml)))

In [11]:
#Question 2.2


triv_model = sm.OLS(y, x_mat).fit()
#triv_model.summary(), triv_model.mse_total

In [12]:
#Test pour hétéroscédasticité
triv_betas = np.array(([triv_model.params[1], triv_model.params[2], triv_model.params[3]]))
triv_res = np.ones(x_mat.shape[0])
for el in np.arange(y.shape[0]):
    triv_res[el] = y[el] - (triv_model.params[0] + triv_betas @ x_mat[el, 1:])
triv_res = triv_res ** 2
#White test
triv_bp = sm.stats.diagnostic.het_breuschpagan(triv_model.resid, x_mat)
triv_bp[0] < stats.chi2.isf(0.05, 3) #on ne peut donc pas rejeter H0

True

In [ ]:
rmw = np.atleast_2d(df[0:200, 3]).T
cma = np.atleast_2d(df[0:200, 4]).T
x_quint = np.hstack((x_mat, rmw, cma))
quint_model = sm.OLS(y, x_quint).fit()
#quint_model.summary(), quint_model.mse_total

In [14]:
#Question 2.6 check for heteroscedasticity in 5 variable model


quint_bp = sm.stats.diagnostic.het_breuschpagan(quint_model.resid, x_quint)
quint_bp[0] < stats.chi2.isf(0.05, 5) #on ne peut pas rejeter h0

True

In [15]:
dt_2.head(5)

,R,NoComp,Industrie,Actifs,Dette,Ventes,CF,CV
0,0.05314,1,1,485.62,335.34,262.44,94.32,157.20
1,0.05449,2,1,227.24,154.36,381.36,266.70,209.55
2,0.05240,3,1,445.31,449.45,1090.25,261.60,283.40
3,0.05621,4,1,346.97,291.48,142.27,46.86,25.56
4,0.03885,5,1,308.67,231.75,621.09,37.26,298.08


In [16]:
int_y = df_2[0:,0]
d_a = np.atleast_2d(np.divide(df_2[0:,4], df_2[0:,3])).T
d_v = np.atleast_2d(np.divide(df_2[0:,4], df_2[0:,5])).T
cf_v = np.atleast_2d(np.divide(df_2[0:,6], df_2[0:,5])).T
cf_a = np.atleast_2d(np.divide(df_2[0:,6], df_2[0:,3])).T
int_x = sm.add_constant(np.hstack((d_a, d_v, cf_v, cf_a)))

In [17]:
#3.1 OLS

debt_r = sm.OLS(int_y, int_x).fit()
#debt_r.summary()